In [1]:
print("Hello World")

Hello World


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki_retrieval = WikipediaQueryRun(api_wrapper=api_wrapper)

wiki_retrieval.name

'wikipedia'

In [3]:
from langchain_chroma.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import chromadb
import os

# Load environment variables
load_dotenv()

# Load document from website
loader = WebBaseLoader("https://panaverse-dao-peach.vercel.app/") 
document = loader.load()

# Split documents
recursive_documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(document)

# Initialize Embedding 
openai_embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# Initialize vector store with client settings
client_settings = chromadb.Settings(persist_directory="./chroma_data")

vector_store = Chroma.from_documents(
    recursive_documents,
    embedding=openai_embedding,
    client_settings=client_settings  # Specify a directory to persist data
)

retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 20, "lambda_mult": 0.5},
)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="panaverse_search",
    description="Search for Information about Panaverse Dao. For any question about Panaverse Dao, you must be used this tool!",
)

retriever_tool.name

'panaverse_search'

In [5]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

arxiv.name

'arxiv'

In [7]:
tools = [wiki_retrieval, retriever_tool, arxiv]

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


In [9]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [10]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [11]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [15]:
from sqlmodel import SQLModel

class Output(SQLModel):
    input: str
    output: str

In [17]:
# Invoke the agent executor and parse the result into the Output model
result_dict = agent_executor.invoke({"input": "What is the purpose of Panaverse Dao?"})
result = Output(**result_dict)

# Access the output
print(result.output)



> Entering new AgentExecutor chain...

Invoking: `panaverse_search` with `{'query': 'purpose of Panaverse Dao'}`




Number of requested results 20 is greater than number of elements in index 7, updating n_results = 7


Panaverse DaoHomeCoursesWeb 3.0 And 
Metaverse

SpecializationSpecialized TrackGenomics and Bioinformatics SpecializationSpecialized TrackNetwork Programmability and Automation SpecializationThe Outcome for Participants of the ProgramThe graduates of this program will own products (Full-Stack App Templates, AR and VR Experiences, and APIs) that are marketed globally by the Panaverse DAO and, if they like, will also be able to start off by offering services at a rate of $50 per hour ($96,000 per year). This would give Pakistani professionals and students a fantastic opportunity to better their financial situation while also giving the economy a much-needed boost by expanding software exports.Top 5 'Metaverse' jobs that will rule the future of tech industryBlockchain Developer Salary - Jun 2022Web3 Salaries Soar to $750,000 for Rank-and-File DevsThe Metaverse, Blockchain Gaming, and NFTs: Navigating the Internet’s Uncharted WatersHow To Become Metaverse Developer: Scope, Skills, and Sala

In [18]:
result_dict = agent_executor.invoke({"input": "What is Machine Learning"})
result = Output(**result_dict)

# Access the output
print(result.output)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks witMachine learning (ML) is a field of study within artificial intelligence that focuses on the development and analysis of statistical algorithms that can learn from data. These algorithms enable systems to improve their performance on tasks through experience, allowing them to generalize from the training data to unseen data.

> Finished chain.
Machine learning (ML) is a field of study within artificial intelligence that focuses on the development and analysis of statistical algorithms that can learn from data. These algorithms enable systems to improve their performance on tasks through experience, allowing them to generalize from the train

In [19]:
result_dict = agent_executor.invoke({"input": "What is the name of 2410.06402 this article?"})
result = Output(**result_dict)

# Access the output
print(result.output)



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '2410.06402'}`


Published: 2024-10-08
Title: Remarks on Lang's Conjectural Characterisation of Hyperbolicity of Projective Manifolds
Authors: Kyle Broder, Frédéric Campana
Summary: S. Lang conjectured that the only obstructions to a projective manifold being
hyperboThe title of the article with the identifier 2410.06402 is "Remarks on Lang's Conjectural Characterisation of Hyperbolicity of Projective Manifolds." It was authored by Kyle Broder and Frédéric Campana.

> Finished chain.
The title of the article with the identifier 2410.06402 is "Remarks on Lang's Conjectural Characterisation of Hyperbolicity of Projective Manifolds." It was authored by Kyle Broder and Frédéric Campana.
